In [1]:
import pandas as pd

In [2]:
from nsepy import get_history as gh
import datetime as dt

In [3]:
start = dt.datetime(2021,6,1)
end = dt.datetime(2022,2,11)
stk_data = gh(symbol='ZOMATO',start=start,end=end)

In [4]:
stk_data=stk_data[["Open","High","Low","Close"]]

In [5]:
stk_data

,Open,High,Low,Close
Date,,,,
2021-07-23,116.00,138.90,115.00,126.00
2021-07-26,126.35,143.75,125.30,140.65
2021-07-27,141.70,147.80,127.75,132.90
2021-07-28,131.00,135.00,123.55,131.20
2021-07-29,134.95,144.00,132.20,141.55
...,...,...,...,...
2022-02-07,95.10,95.50,93.35,93.55
2022-02-08,94.35,94.65,91.25,93.55
2022-02-09,94.50,95.45,93.60,94.20


In [6]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (140, 4)


In [7]:
data1

array([[0.35683761, 0.60653595, 0.44008559, 0.52094972],
       [0.5042735 , 0.66993464, 0.58701854, 0.72555866],
       [0.72293447, 0.72287582, 0.62196862, 0.61731844],
       [0.57051282, 0.55555556, 0.56205421, 0.59357542],
       [0.62678063, 0.67320261, 0.68544936, 0.73812849],
       [0.73575499, 0.65620915, 0.66833096, 0.62569832],
       [0.63817664, 0.63071895, 0.7275321 , 0.7122905 ],
       [0.65598291, 0.63137255, 0.75392297, 0.70810056],
       [0.69586895, 0.63398693, 0.72895863, 0.69413408],
       [0.68091168, 0.60653595, 0.68259629, 0.64594972],
       [0.63461538, 0.57124183, 0.65549215, 0.59567039],
       [0.59045584, 0.53660131, 0.61483595, 0.58519553],
       [0.57051282, 0.50915033, 0.54136947, 0.50977654],
       [0.45655271, 0.60457516, 0.55420827, 0.65572626],
       [0.63675214, 0.5869281 , 0.68330956, 0.65293296],
       [0.61111111, 0.61764706, 0.68402282, 0.67946927],
       [0.6474359 , 0.58039216, 0.68616262, 0.64594972],
       [0.59615385, 0.54705882,

In [8]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])
data1

,Open,High,Low,Close
0,0.356838,0.606536,0.440086,0.520950
1,0.504274,0.669935,0.587019,0.725559
2,0.722934,0.722876,0.621969,0.617318
3,0.570513,0.555556,0.562054,0.593575
4,0.626781,0.673203,0.685449,0.738128
...,...,...,...,...
135,0.059117,0.039216,0.131241,0.067737
136,0.048433,0.028105,0.101284,0.067737
137,0.050570,0.038562,0.134807,0.076816
138,0.056268,0.037908,0.126248,0.080307


In [11]:
training_size = round(len(data1 ) * 0.80)
print("Training size:",training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

Training size: 112
X_train length: (112, 4)
X_test length: (28, 4)
y_train length: (112, 4)
y_test length: (28, 4)


In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [38]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    print('DatasetTwo', datasetTwo)
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    print('Train length', len(train))
    print('Test length', len(test))
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Results: ', results)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [39]:
listt=["Close","High"]
listt

['Close', 'High']

In [40]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High']
DatasetTwo         Close      High
0    0.520950  0.606536
1    0.725559  0.669935
2    0.617318  0.722876
3    0.593575  0.555556
4    0.738128  0.673203
..        ...       ...
135  0.067737  0.039216
136  0.067737  0.028105
137  0.076816  0.038562
138  0.080307  0.037908
139  0.000000  0.003268

[140 rows x 2 columns]
Train length 112
Test length 28
Results:  <statsmodels.tsa.vector_ar.var_model.VARResultsWrapper object at 0x000001E95797FA48>
Order = 1
AIC:  -12.802906462354514
BIC:  -12.656445370391685

Results:  <statsmodels.tsa.vector_ar.var_model.VARResultsWrapper object at 0x000001E957A46248>
Order = 2
AIC:  -12.939334813997244
BIC:  -12.693836598925206

Results:  <statsmodels.tsa.vector_ar.var_model.VARResultsWrapper object at 0x000001E957A466C8>
Order = 3
AIC:  -13.023577564229878
BIC:  -12.677899854585768

Results:  <statsmodels.tsa.vector_ar.var_model.VARResultsWrapper object at 0x000001E957A46908>
Order = 4
AIC:  -12.949471100557602
BIC:  -12.502449229370

In [29]:
data1

,Open,High,Low,Close
0,0.356838,0.606536,0.440086,0.520950
1,0.504274,0.669935,0.587019,0.725559
2,0.722934,0.722876,0.621969,0.617318
3,0.570513,0.555556,0.562054,0.593575
4,0.626781,0.673203,0.685449,0.738128
...,...,...,...,...
135,0.059117,0.039216,0.131241,0.067737
136,0.048433,0.028105,0.101284,0.067737
137,0.050570,0.038562,0.134807,0.076816
138,0.056268,0.037908,0.126248,0.080307


In [30]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0.0,1.115900e+14,2,28
1,"[Close, High, Open]",0.0,1.096371e+14,1,28
2,"[Close, High]",0.0,1.094502e+14,1,28


In [31]:
performance

{'Model': [['Close', 'High', 'Open', 'Low'],
  ['Close', 'High', 'Open'],
  ['Close', 'High']],
 'RMSE': [0.0, 0.0, 0.0],
 'MaPe': [111589951338041.5, 109637093974948.62, 109450194907937.44],
 'Lag': [2, 1, 1],
 'Test': [28, 28, 28]}